In [2]:
import os,sys
import warnings
import pandas as pd
import statistics
import numpy as np

import matplotlib
from matplotlib        import pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
# -------------------------------------------------------------------------------- # sklearn Modules
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import HuberRegressor, LinearRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
# -------------------------------------------------------------------------------- # Custom Modules 
from src import helper
from src import cluster

plt.style.use('ggplot')
RUN_TYPE = "Colab"
y_cols = ["gross", "budget"]
X_col = ["rating", "score"] # "genre","company"]
cmd_movie = "./"
h = 0.02  # step size in the mesh



def cleanLabels(df, col_name):
    avg_gross = helper.mean(df[col_name].to_numpy())
    _data = df[col_name].to_numpy()
    for index,_ in enumerate(_data):
        if _data[index] >= avg_gross:
            _data[index] = 3
        elif _data[index] >= (avg_gross - avg_gross/2):
            _data[index] = 2
        elif _data[index] >= avg_gross/2:
            _data[index] = 1
        else:
            _data[index] = 0
    df[col_name] = _data
    return df


def clean_df(_DATA_):
    _DATA_ = helper.removeNaN(_DATA_, "gross")
    _DATA_ = helper.removeNaN(_DATA_, "budget")
    _DATA_ = helper.removeNaN(_DATA_, "rating")
    _DATA_ = helper.removeNaN(_DATA_, "score")   # company

    _DATA_ = cleanLabels(_DATA_, "gross")
    _DATA_ = cleanLabels(_DATA_, "budget")



    df_rating = [str(i) for i in _DATA_.rating.values]
    df_genre   = [str(i) for i in _DATA_.genre.values]
    df_company = [str(i) for i in _DATA_["company"].values]


    ratingLT = list(set(df_rating))
    companyLT = list(set(df_company))
    genreLT   = list(set(df_genre))


    for idx, row in enumerate(_DATA_.rating):
        df_rating[idx] = ratingLT.index(row)

    for idx, row in enumerate(_DATA_.genre):
        df_genre[idx] = genreLT.index(row)

    for idx, row in enumerate(df_company):
        df_company[idx] = companyLT.index(row)



    _DATA_["rating"] = df_rating
    _DATA_["genre"] = df_genre
    _DATA_["company"] = df_company
    return _DATA_


def runClustering(DATA):
    for y_col in y_cols[0]:
        # print (f"Generating KNN with\n   ==> label_col = {y_col}")
        y = DATA[y_col].to_numpy()
        X = DATA[X_col].to_numpy()
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=42)
        KNN = cluster.train(X, y)
        # y_pred = KNN.predict(X_test)
        # crossValResults = cluster.Eval.cross_val(KNN, X_test, y_test)

        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                             np.arange(y_min, y_max, h))
        zz = np.c_[xx.ravel(), yy.ravel()]
        print(xx.ravel().shape, yy.ravel().shape)
        print(zz.shape)
        print(X.shape, y.shape)
        Z = KNN.predict(zz)

        
        # Put the result into a color plot
        Z = Z.reshape(xx.shape)
        plt.figure(figsize=(8, 6))
        plt.contourf(xx, yy, Z, cmap=cmap_light)

        # Plot also the training points
        sns.scatterplot(
            x=X[:, 0],
            y=X[:, 1],
            hue=iris.target_names[y],
            palette=cmap_bold,
            alpha=1.0,
            edgecolor="black",
        )
        plt.xlim(xx.min(), xx.max())
        plt.ylim(yy.min(), yy.max())
        plt.title(
            "3-Class classification (k = %i, weights = '%s')" % (n_neighbors, weights)
        )
        plt.xlabel(iris.feature_names[0])
        plt.ylabel(iris.feature_names[1])
        plt.show()



def main():

    DATA_PATH = "data/movies_v1.csv"
    DATA_ORIGINAL_PATH = "data/movies_original.csv"
    cwd = os.getcwd()

    DATA_PATH = os.path.join(cmd_movie, DATA_PATH)
    DATA_ORIGINAL_PATH = os.path.join(cmd_movie, DATA_ORIGINAL_PATH)

    # print(DATA_PATH)
    DATA = pd.read_csv(DATA_PATH)
    DATA_ORIGINAL = pd.read_csv(DATA_ORIGINAL_PATH)
    # ------------------------------------------------------------ #
    print("Booting up the Movie Recommended System ...")
    # print (f"{DATA} ")
    # print (f"{DATA_ORIGINAL} ")
    runClustering(clean_df(DATA))

    exit()

    # exit()
    # print("--------------------------------")
    # print (f"DATA -- ORIGINAL")
    # # KNN, X_test, y_test = runClustering(clean_df(DATA_ORIGINAL))


if __name__ == "__main__":
    # DATA_PATH_original = "data/movies_original.csv"
    main()
    # y = df_clean[y_col[0]].to_numpy()
    # X = df_clean[X_col].to_numpy()
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)
    # knn = trainCluster(X,y)
    # modelEval(knn, X_test, y_test)


Booting up the Movie Recommended System ...
Remove Nan from DF
                                  name rating     genre  year  \
0                           Maleficent     PG    Action  2014   
1            Pokémon Detective Pikachu     PG    Action  2019   
2                         TRON: Legacy     PG    Action  2010   
3                       The Karate Kid     PG    Action  2010   
4     Journey 2: The Mysterious Island     PG    Action  2012   
...                                ...    ...       ...   ...   
1347          Star Trek First Frontier    NaN    Sci-Fi  2020   
1348                     Bennett's War  PG-13     Sport  2019   
1349                      No Good Deed  PG-13  Thriller  2014   
1350                      The Intruder  PG-13  Thriller  2019   
1351                 The Boy Next Door      R  Thriller  2015   

                released  score     votes          director  \
0           May 30, 2014    7.0  353000.0  Robert Stromberg   
1           May 10, 2019    6.

KeyError: 'g'

In [2]:

        # 
        # Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

        # # Put the result into a color plot
        # Z = Z.reshape(xx.shape)

        # exit()
        # print(f"{y_col} -- {helper.mean(crossValResults)}")
        # cluster.generateFigure(KNN, X_test, y_test)
        # return KNN, X_test, y_test




